### **03_rag.ipynb**
### **Retrieval-augmented generation**

* ##### 01 - Install packages
* ##### 02 - Import packages
* ##### 03 - Download Multilingual Universal Sentence Encoder model
* ##### 04 - Create Elasticsearch client
* ##### 05 - Create Elasticsearch vector store
* ##### 06 - Download Large Language Model
* ##### 07 - Question answering

### 01 - Install packages

In [ ]:
import sys

In [ ]:
!{ sys.executable } -m pip install --upgrade elasticsearch huggingface-hub langchain langchain-community llama-cpp-python tensorflow tensorflow-hub tensorflow-text urllib3

### 02 - Import packages

In [ ]:
from elasticsearch          import Elasticsearch
from huggingface_hub        import hf_hub_download
from langchain.chains       import RetrievalQA
from langchain.embeddings   import TensorflowHubEmbeddings
from langchain.llms         import LlamaCpp
from langchain.vectorstores import ElasticsearchStore

In [ ]:
from urllib3 import disable_warnings
disable_warnings()

### 03 - Download Multilingual Universal Sentence Encoder model

In [ ]:
embedding = TensorflowHubEmbeddings(model_url = 'https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/multilingual-large/versions/2')

In [ ]:
embedding.embed_query('Hello World, ML Elasticsearch!')

### 04 - Create Elasticsearch client

In [ ]:
es_host     = '<elasticsearch_host>'
es_username = '<elasticsearch_username>'
es_password = '<elasticsearch_password>'

In [ ]:
es = Elasticsearch(
    hosts        = es_host,
    basic_auth   = (es_username, es_password),
    verify_certs = False
)

In [ ]:
es.info()

### 05 - Create Elasticsearch vector store

In [ ]:
bbc_news_index = 'bbc_news'

In [ ]:
vector_store = ElasticsearchStore(
    es_connection = es,
    embedding     = embedding,
    index_name    = bbc_news_index
)

In [ ]:
vector_store.similarity_search('ibm')[0].page_content

### 06 - Download Large Language Model

In [ ]:
llm_path = hf_hub_download('afrideva/Tiny-Vicuna-1B-GGUF', filename = 'tiny-vicuna-1b.q4_k_m.gguf')
llm      = LlamaCpp(model_path = llm_path, n_ctx = 4096, max_tokens = 256)

In [ ]:
llm('What is IBM?')

### 07 - Question answering

In [ ]:
retrieval_qa = RetrievalQA.from_llm(llm, retriever = vector_store.as_retriever(), return_source_documents = True)

In [ ]:
def qa(question):

    answer = retrieval_qa({ 'query' : question })

    print(f"Generated answer : { answer['result'] }")
    print(f"Source answer    : { answer['source_documents'][0].page_content }")

In [ ]:
qa('What is IBM doing for open source?')

In [ ]:
qa('¿Qué está haciendo IBM por el código abierto?')

In [ ]:
qa('O que a IBM está fazendo pelo código aberto?')